In [ ]:
import json
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import re
import pandas as pd
import glob
import seaborn as sns
from collections import defaultdict
sns.set_theme()

In [ ]:
TGT="de"
DATA = "mustc"
SPLIT = "dev"
SPLIT = "tst-COMMON"
RESULT = Path(f"{DATA}_{TGT}_{SPLIT}-results")
MODELS = [
    f"mma_{TGT}_hard_aligned",
    f"mma_{TGT}_infinite_lookback",
    f"cif_{TGT}_align_ctc0_3_lat",
    f"cif_{TGT}_il_sum_ctc0_3_lat",
]
OUTPUT = Path(f"./graphs/")
OUTPUT.mkdir(parents=True, exist_ok=True)
SORTBY = "AL"

data = defaultdict(list)

for m in MODELS:
    for p in glob.glob(f"{RESULT}/{m}*"):

        dat = json.load(open(f"{p}/scores"))
        
        data[m].append({
            "BLEU": dat["Quality"]["BLEU"],
            "AL": dat["Latency"]["AL"],
            "DAL": dat["Latency"]["DAL"],
#             "AL_CA": dat["Latency"]["AL_CA"]
        })
        
    data[m] = sorted(data[m], key=lambda x: x[SORTBY])

In [ ]:
pd.DataFrame.from_dict(data, orient='index').round(2)

In [ ]:
formal = {
    "BLEU": "BLEU",
    "AL": "AL (ms)", 
    "DAL": "DAL (ms)",
}
MODELNAMES = [
    "MMA-H",
    "MMA-IL",
    "CIF-F", 
    "CIF-IL", 
#     "MMA-H (CA)",
#     "MMA-IL (CA)",
#     "CIF-F (CA)", 
#     "CIF-IL (CA)", 
]
STYLES = [
    "^-b",
    "s-r",
    "p-m",
    "*-y",
    "^:b",
    "s:r",
    "p:m",
    "*:y",
]


nmodel = len(MODELS)
BLEUs = {
    m: [ d["BLEU"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}
ALs = {
    m: [ d["AL"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}
DALs = {
    m: [ d["DAL"] for d in data[MODELS[i % nmodel]] ]
    for i,m in enumerate(MODELNAMES)
}
# AL_CAs = {
#     m: [ d["AL_CA"] for d in data[MODELS[i]] ]
#     for i,m in enumerate(MODELNAMES)
# }


BLEUs

In [ ]:
def plot(x, y, xlabel, ylabel, names=MODELNAMES, styles=STYLES, legend=False, file=None):
    figsize=(3, 3)
    dpi=200

    fig = plt.figure(figsize=figsize, dpi=dpi)
    ax = fig.add_subplot(111)

    for i,m in enumerate(names):
        ax.plot(x[m], y[m], styles[i%len(styles)], label=m, markersize=6 if "(CA)" in m else 8)

    if legend:
        ax.legend() # loc='upper right'
    ax.autoscale()

    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    fig.tight_layout()
    fig.show()
    if file is not None:
        plt.savefig(
            file if isinstance(file, str) else str(file.as_posix()),
            dpi=dpi,
            bbox_inches='tight', pad_inches=0)

# plot(ALs, BLEUs, formal["AL"], formal["BLEU"], legend=True)
plot(ALs, BLEUs, "", formal["BLEU"], file=f"result_AL_{SPLIT}.pdf")

In [ ]:
plot(DALs, BLEUs, "", "", file=f"result_DAL_{SPLIT}.pdf")

In [ ]:
plot(DALs, BLEUs, "", "", legend=True, file=f"result_DAL_{SPLIT}.pdf")

In [ ]:
RESULT = Path(f"{DATA}_{TGT}_{SPLIT}_long-results")

for m in MODELS:
    mlong = f"{m}(l)"
    for p in glob.glob(f"{RESULT}/{m}*"):

        dat = json.load(open(f"{p}/scores"))
        
        data[mlong].append({
            "BLEU": dat["Quality"]["BLEU"],
            "AL": dat["Latency"]["AL"],
            "DAL": dat["Latency"]["DAL"],
        })
        
    data[mlong] = sorted(data[mlong], key=lambda x: x[SORTBY])

MODELNAMES2 = [
    "MMA-H(l)",
    "MMA-IL(l)",
    "CIF-F(l)", 
    "CIF-IL(l)",
]

BLEUs.update({
    m: [ d["BLEU"] for d in data[f"{MODELS[i % nmodel]}(l)"] ]
    for i,m in enumerate(MODELNAMES2)
})
ALs.update({
    m: [ d["AL"] for d in data[f"{MODELS[i % nmodel]}(l)"] ]
    for i,m in enumerate(MODELNAMES2)
})
DALs.update({
    m: [ d["DAL"] for d in data[f"{MODELS[i % nmodel]}(l)"] ]
    for i,m in enumerate(MODELNAMES2)
})

In [ ]:
plot(ALs, BLEUs, formal["AL"], formal["BLEU"], MODELNAMES+MODELNAMES2, legend=True)

In [ ]:
BLEUs


In [ ]:
best_i = {m: np.argmax(BLEUs[m]) for m in MODELNAMES}
best_record = {m: (BLEUs[f"{m}(l)"][best_i[m]], BLEUs[m][best_i[m]]) for m in MODELNAMES}
{k: f"{a-b:.02f}" for k, (a, b) in best_record.items()}

In [ ]:
best_record

In [ ]:
def mean(x):
    return sum(x) / len(x)
{m: f'{mean([a-b for a, b in zip(BLEUs[f"{m}(l)"], BLEUs[m])]):.02f}' for m in MODELNAMES}

In [ ]:
import sys
sys.path.insert(0, "/home/george/Projects/simulst/DATA")
from mustc.prep_mustc_data import (
    MUSTC
)
dataset = MUSTC("/media/george/Data/mustc", "de", "dev")
durations = []
for wav_path, offset, n_frames, sr, src_utt, tgt_utt, spk_id, utt_id in dataset.data:
    durations.append(n_frames / sr)
plt.hist(durations)

In [ ]:
for c in [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]:
    num = len([d for d in durations if d > c])
    print(c, ":", num, f"({num / len(durations) * 100: .02f}%)")

In [ ]:
sacrebleu.corpus_bleu

In [ ]:
import sacrebleu
with open("data_de/dev.de", "r") as f:
    ref_dev = [line.strip() for line in f]
with open("data_de/tst-COMMON.de", "r") as f:
    ref_tst = [line.strip() for line in f]
    
models=[
    "mma_de_hard_aligned_0_2",
    "mma_de_infinite_lookback_0_2",
    "cif_de_align_ctc0_3_lat1_0",
    "cif_de_il_sum_ctc0_3_lat1_0"
]
    
preds_dev = {}
for m in models:
    with open(f"mustc_de_dev-results/{m}/prediction", "r") as f:
        preds_dev[m] = [line.strip() for line in f]
preds_tst = {}
for m in models:
    with open(f"mustc_de_tst-COMMON-results/{m}/prediction", "r") as f:
        preds_tst[m] = [line.strip() for line in f]

print("full")
for m in preds_dev:
    ref = ref_dev + ref_tst
    hyp = preds_dev[m] + preds_tst[m]
    print(f"{sacrebleu.corpus_bleu(hyp, [ref]).score:.02f}")
print("20s")
for m in preds_dev:
    ref = ref_dev[-40:] + ref_tst[-47:]
    hyp = preds_dev[m][-40:] + preds_tst[m][-47:]
    print(f"{sacrebleu.corpus_bleu(hyp, [ref]).score:.02f}")